In [ ]:
from astropy.table import Table
from astropy import units as u
from astropy.coordinates import SkyCoord

import os.path as op

import numpy as np
import pandas as pd

In [ ]:
# Load HETDEX
path_to_cat = '/home/alberto/almacen/HETDEX_catalogs/hetdex_source_catalog_1'
version = 'v3.2'

hetdex_table = pd.DataFrame(
    dict(Table.read(op.join(path_to_cat, f'hetdex_sc1_{version}.ecsv')))
    )

# Load PAUS selection
load_path = '/home/alberto/almacen/PAUS_data/catalogs'
selection = pd.read_csv(f'{load_path}/LAE_selection.csv')

selection[np.isnan(selection)] = -1

selection

In [ ]:
hetdex_table

In [ ]:
# Xmatch
coords_paus = SkyCoord(ra=np.array(selection['RA']) * u.deg,
                       dec=np.array(selection['DEC']) * u.deg)
coords_hetdex = SkyCoord(ra=np.array(hetdex_table['RA']) * u.deg,
                         dec=np.array(hetdex_table['DEC']) * u.deg)
                    
xm_id, ang_dist, _= coords_paus.match_to_catalog_sky(coords_hetdex)

# Objects with 1 arcsec of separation
mask_dist = (ang_dist <= 1 * u.arcsec)

selection['z_HETDEX'] = np.array(hetdex_table['z_hetdex'])[xm_id]
selection['z_HETDEX'][~mask_dist] = np.ones(sum(~mask_dist)) * -1

selection

In [ ]:
sum(selection['z_HETDEX'] > 0)